Задание:
"Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Надо выбрать 20 таких локаций для размещения, чтобы польза была большой.
Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры.Также эти места должны быть популярны среди туристов.
Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare".

Часть открытых данных была скачана в формате dat, для дальнейшей работы преобразуем данные в формат csv.

In [7]:
import csv

#конвертация файла из формата dat в csv с разделением по "|" и удалением строк с недостающими данными
with open('checkins.dat') as datfile:
    with open('checkins2.csv','w') as bannerfile:
        csv_writer = csv.writer(bannerfile)
        i=0
        for line in datfile:
            row = [field.strip() for field in line.split('|')]
            if len(row) == 6 and row[3] and row[4]:
                csv_writer.writerow(row)
                i+=1

Откроем преобразованный файл на чтение и отобразим первые 5 строк.

In [8]:
import pandas as pd
data=pd.read_csv('checkins2.csv', sep=',')
data.head(5)

,id,user_id,venue_id,latitude,longitude,created_at
0,984222,15824,5222,38.895112,-77.036366,2012-04-21 17:43:47
1,984234,44652,5222,33.800745,-84.410520,2012-04-21 17:43:43
2,984291,105054,5222,45.523452,-122.676207,2012-04-21 17:39:22
3,984318,2146539,5222,40.764462,-111.904565,2012-04-21 17:35:46
4,984232,93870,380645,33.448377,-112.074037,2012-04-21 17:38:18


Далее посмотрим на информацию о данных.

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396634 entries, 0 to 396633
Data columns (total 6 columns):
id            396634 non-null int64
user_id       396634 non-null int64
venue_id      396634 non-null int64
latitude      396634 non-null float64
longitude     396634 non-null float64
created_at    396634 non-null object
dtypes: float64(2), int64(3), object(1)
memory usage: 18.2+ MB


Согласно информации выше, пропущенных данных в файле нет, но на всякий случай убедимся в этом.

In [10]:
data.isnull().values.any()

False

В целях упрощения обработки информации на данном компьютере, уменьшим количество строк данных до 100 000.

In [11]:
datacruise=data[:100000]

Для выбора локаций, нас интересуют колонки с координатами (широта и долгота).

In [12]:
columns=['latitude','longitude']
datacruise=datacruise[columns]
datacruise.head(5)

,latitude,longitude
0,38.895112,-77.036366
1,33.800745,-84.410520
2,45.523452,-122.676207
3,40.764462,-111.904565
4,33.448377,-112.074037


Чтобы выявить кластеры, воспользуемся алгоритмом MeanShift, указав ограничение для размера кластеров bandwidth=0.1 (примерно от 5 до 10 км).

In [13]:
from sklearn.cluster import MeanShift
ms=MeanShift(bandwidth=0.1)
ms.fit(datacruise) #обучим алгоритм на выбранных данных

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=1, seeds=None)

Посмотрим сколько всего кластеров получилось, обозначим уникальные кластеры и их центры.

In [20]:
import numpy as np
labels = ms.labels_
cluster_centers = ms.cluster_centers_
labels_unique = np.unique(labels)
n_clusters = len(labels_unique)
print 'Количество кластеров: {}'.format(n_clusters)

Количество кластеров: 3230


Посчитаем количество элементов в каждом кластере и преобразуем в словарь.

In [26]:
import collections
c=collections.Counter(labels)
labelnew=dict(c)

Соединим в одну матрицу 4 столбца: номер кластера, количество элементов в нем, широта и долгота.

In [27]:
L=np.zeros((3230,4))
i=0
for line in cluster_centers:
    a=labelnew[i]
    L[i:]=(int(i),a,cluster_centers[i,0],cluster_centers[i,1])
    i+=1

Согласно пояснению: "Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения."

Оставим кластеры, в которых более 15-ти элементов. И посмотрим сколько таких кластеров.

In [42]:
K=np.zeros((sum(L[:,1]>15),4))
i=0
j=0
for i in range(0,len(L)):
    p=L[i,1]
    if p>15:        
        K[j:]=(L[i,0],L[i,1],L[i,2],L[i,3])
        j+=1
    i+=1    

In [43]:
len(K)

593

Согласно дополнению: "При желании увидеть получившиеся результаты на карте можно передать центры получившихся кластеров в один из инструментов визуализации. Например, сайт mapcustomizer.com имеет функцию Bulk Entry, куда можно вставить центры полученных кластеров".

Запишем в файл csv получившиеся центры кластеров.
       

In [70]:
i=0        
with open('formap.csv','w') as mapp:
    for i in range(0,len(K)):
        line=','.join([str(K[i,2]),str(K[i,3])])
        mapp.write(line+'\n')
        i+=1

По условию 20 баннеров надо разместить близ офисов компании. Координаты офисов также даны.

Посчитаем расстояния от данных офисов до каждого центра полученных кластеров.

In [71]:
import scipy

i=0
k=0

Distance=np.zeros((len(K)*6,4))

office1=np.array((33.751277,-118.188740)) # координаты офиса в Los Angeles
office2=np.array((25.867736,-80.324116)) # координаты офиса в Miami
office3=np.array((51.503016,-0.075479)) # координаты офиса в London
office4=np.array((52.378894,4.885084)) # координаты офиса в Amsterdam
office5=np.array((39.366487,117.036146)) # координаты офиса в Beijing
office6=np.array((-33.868457,151.205134)) # координаты офиса в Sydney

for i in range(0,len(K)):
    
    dist1=scipy.spatial.distance.euclidean(office1, K[i,2:4])
    dist2=scipy.spatial.distance.euclidean(office2, K[i,2:4])
    dist3=scipy.spatial.distance.euclidean(office3, K[i,2:4])
    dist4=scipy.spatial.distance.euclidean(office4, K[i,2:4])
    dist5=scipy.spatial.distance.euclidean(office5, K[i,2:4])
    dist6=scipy.spatial.distance.euclidean(office6, K[i,2:4])
    
    Distance[k:]=(K[i,0],dist1,K[i,2],K[i,3])
    k+=1
    Distance[k:]=(K[i,0],dist2,K[i,2],K[i,3])
    k+=1
    Distance[k:]=(K[i,0],dist3,K[i,2],K[i,3])
    k+=1
    Distance[k:]=(K[i,0],dist4,K[i,2],K[i,3])
    k+=1
    Distance[k:]=(K[i,0],dist5,K[i,2],K[i,3])
    k+=1
    Distance[k:]=(K[i,0],dist6,K[i,2],K[i,3])
    
    k+=1
    i+=1

Отсортируем кластеры по удаленности от офисов. И первые 20 баннеров как раз и нужно будет разместить, согласно указанным координатам.

In [73]:
Distance = pd.DataFrame(Distance)
Distance = Distance.rename(columns={0:'№ of cluster', 1:'Distance', 2:'Latitude', 3:'Longitude'})
Distance_sorted = Distance.sort_values('Distance')
Distance_sorted.head(20)

,№ of cluster,Distance,Latitude,Longitude
2465,413.0,0.007835,-33.860630,151.204776
2235,373.0,0.009353,52.372964,4.892317
2413,405.0,0.022674,25.845672,-80.318891
350,58.0,0.050058,51.502991,-0.125537
306,51.0,0.070848,33.809878,-118.148924
175,29.0,0.134109,25.785812,-80.217938
997,166.0,0.167406,25.705350,-80.283429
553,92.0,0.188876,26.010098,-80.199991
522,87.0,0.195779,33.888325,-118.048928
252,42.0,0.211811,33.872986,-118.362091
